# GDP

Notebook environment to migrate netcdf files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))
sys.path.append(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\coclicodata')

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = p_drive.joinpath("1000545-054-globalbeaches", "15_GlobalCoastalAtlas", "datasets")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

C:\Users\heijd_de\AppData\Local\Temp\ipykernel_10416\2707390607.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


<IPython.core.display.Javascript object>

In [3]:
# Project paths & files (manual input)
dataset_dir = gca_data_dir.joinpath("06_Worldgdp")
dataset_dir_gdp = dataset_dir.joinpath("world_gdp.nc")
dataset_out_file = "world_gdp"
CF_dir = gca_data_dir.joinpath(r"CF")  # directory to save output CF check files

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [4]:
# open datasets
ds_gdp = xr.open_dataset(dataset_dir_gdp)

# check original dataset
ds_gdp

<xarray.Dataset>
Dimensions:               (index: 1723302)
Coordinates:
  * index                 (index) int64 0 1 2 3 ... 1739823 1739824 1739825
Data variables: (12/20)
    transect_id           (index) object ...
    country_id            (index) object ...
    continent             (index) object ...
    country_name          (index) object ...
    Start_lon             (index) float64 ...
    Start_lat             (index) float64 ...
    ...                    ...
    roads                 (index) float64 ...
    railways              (index) float64 ...
    ports                 (index) float64 ...
    airports              (index) float64 ...
    gdp_ppp_usd2017_2015  (index) float64 ...
    gdp_ppp_grid_2015     (index) float64 ...

<IPython.core.display.Javascript object>

In [5]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=dataset_dir_gdp, 
                 working_dir=CF_dir
                 )


<IPython.core.display.Javascript object>

In [6]:
# save original CF compliancy
save_compliancy(cap, testfile=dataset_dir_gdp, working_dir=CF_dir)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\06_Worldgdp\\world_gdp.nc': {'warnings': '27', 'errors': '4'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [7]:
import json

# NetCDF attribute alterations
f_global = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\06_Worldgdp\metadata_worldgdp.json')
meta_global = json.load(f_global)

for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    ds_gdp.attrs[attr_name] = attr_val

ds_gdp.attrs['Conventions'] = "CF-1.8"

<IPython.core.display.Javascript object>

In [8]:
ds_gdp

<xarray.Dataset>
Dimensions:               (index: 1723302)
Coordinates:
  * index                 (index) int64 0 1 2 3 ... 1739823 1739824 1739825
Data variables: (12/20)
    transect_id           (index) object ...
    country_id            (index) object ...
    continent             (index) object ...
    country_name          (index) object ...
    Start_lon             (index) float64 ...
    Start_lat             (index) float64 ...
    ...                    ...
    roads                 (index) float64 ...
    railways              (index) float64 ...
    ports                 (index) float64 ...
    airports              (index) float64 ...
    gdp_ppp_usd2017_2015  (index) float64 ...
    gdp_ppp_grid_2015     (index) float64 ...
Attributes: (12/21)
    TITLE:               World Gridded GDP
    TITLE_ABBREVIATION:  world_gdp
    DESCRIPTION:         The World Gridded GDP dataset contatins GDP per capi...
    SHORT_DESCRIPTION:   The World Gridded GDP dataset contatins GDP per capi...
    INSTITUTION:         Aalto University
    PROVIDERS:           {"name": "Aalto University", "url": "www.aalto.fi/en...
    ...                  ...
    DOI:                 https://doi.org/10.1038/sdata.2018.4
    LONG_NAME:           WORLD_GRIDDED_GDP
    UNITS:               
    COMMENT:             This dataset is modified from the original Kummu et ...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [9]:
# combine start and end coordinates into a transect
from shapely.geometry import LineString

start_lons = ds_gdp["Start_lon"].values
start_lats = ds_gdp["Start_lat"].values
end_lons = ds_gdp["End_lon"].values
end_lats = ds_gdp["End_lat"].values
coords = zip(zip(start_lons, start_lats), zip(end_lons, end_lats))

ds_gdp["transect_geom"] = (
    ["index"],
    [str(LineString(line)) for line in coords],
)
ds_gdp["transect_geom"].attrs["long_name"] = "Transect Geometry"

<IPython.core.display.Javascript object>

In [10]:
keep_vars = ['transect_id', 'country_name', 'continent', 'Start_lon', 'Start_lat', 'transect_geom', 'gdp_ppp_grid_2015']
allvars = list(ds_gdp.keys())
delete_vars = list(set(allvars).difference(set(keep_vars)))

ds_gdp = ds_gdp.drop(delete_vars)

<IPython.core.display.Javascript object>

In [11]:
ds_gdp

<xarray.Dataset>
Dimensions:            (index: 1723302)
Coordinates:
  * index              (index) int64 0 1 2 3 ... 1739822 1739823 1739824 1739825
Data variables:
    transect_id        (index) object ...
    continent          (index) object ...
    country_name       (index) object ...
    Start_lon          (index) float64 -74.39 -74.38 -74.38 ... 39.94 39.94
    Start_lat          (index) float64 -50.38 -50.38 -50.38 ... 64.7 64.7 64.7
    gdp_ppp_grid_2015  (index) float64 ...
    transect_geom      (index) <U92 'LINESTRING (-74.3863095545 -50.377658945...
Attributes: (12/21)
    TITLE:               World Gridded GDP
    TITLE_ABBREVIATION:  world_gdp
    DESCRIPTION:         The World Gridded GDP dataset contatins GDP per capi...
    SHORT_DESCRIPTION:   The World Gridded GDP dataset contatins GDP per capi...
    INSTITUTION:         Aalto University
    PROVIDERS:           {"name": "Aalto University", "url": "www.aalto.fi/en...
    ...                  ...
    DOI:                 https://doi.org/10.1038/sdata.2018.4
    LONG_NAME:           WORLD_GRIDDED_GDP
    UNITS:               
    COMMENT:             This dataset is modified from the original Kummu et ...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [12]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
ds_gdp = ds_gdp.rename_dims({"index": "stations"})

f_vars = open(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\datasets\06_Worldgdp\vars_gdp.json')
meta_vars = json.load(f_vars)

for var_name, var_dict in meta_vars.items():
    ds_gdp = ds_gdp.rename_vars({var_name : var_dict['name']})
    for key, value in var_dict.items():
        if key != 'name':
            ds_gdp[var_dict['name']].attrs[key] = value


# change dtypes
object_vars = ['transect_id', 'country', 'continent', 'transect_geom']
for ov in object_vars:
    ds_gdp[ov] = ds_gdp[ov].astype('S')


all_vars = list(ds_gdp.keys())
data_vars = ['gdp']
# set some data variables to coordinates to avoid duplication of dimensions in later stage
ds_gdp = ds_gdp.set_coords([v for v in all_vars if v not in data_vars])

# drop index xarray
ds_gdp = ds_gdp.drop('index')

<IPython.core.display.Javascript object>

In [13]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first
# dataset["scenarios"]
ds_gdp

<xarray.Dataset>
Dimensions:        (stations: 1723302)
Coordinates:
    transect_id    (stations) |S15 b'BOX_028_183_0' ... b'BOX_211_067_153'
    continent      (stations) |S23 b'South America' ... b'Europe'
    country        (stations) |S40 b'Chile' b'Chile' ... b'Russia' b'Russia'
    lon            (stations) float64 -74.39 -74.38 -74.38 ... 39.94 39.94 39.94
    lat            (stations) float64 -50.38 -50.38 -50.38 ... 64.7 64.7 64.7
    transect_geom  (stations) |S92 b'LINESTRING (-74.3863095545 -50.377658945...
Dimensions without coordinates: stations
Data variables:
    gdp            (stations) float64 ...
Attributes: (12/21)
    TITLE:               World Gridded GDP
    TITLE_ABBREVIATION:  world_gdp
    DESCRIPTION:         The World Gridded GDP dataset contatins GDP per capi...
    SHORT_DESCRIPTION:   The World Gridded GDP dataset contatins GDP per capi...
    INSTITUTION:         Aalto University
    PROVIDERS:           {"name": "Aalto University", "url": "www.aalto.fi/en...
    ...                  ...
    DOI:                 https://doi.org/10.1038/sdata.2018.4
    LONG_NAME:           WORLD_GRIDDED_GDP
    UNITS:               
    COMMENT:             This dataset is modified from the original Kummu et ...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [33]:
# Drop the NaN indices in gdp variable
nans = [i for i, x_g in enumerate(ds_gdp['gdp'].values) if pd.isna(x_g)]
ds_gdp = ds_gdp.isel(stations=~ds_gdp.stations.isin(nans))


<IPython.core.display.Javascript object>

In [35]:
# save new .nc files
ds_gdp.to_netcdf(path=str(dataset_dir_gdp).replace(".nc", "_CF.nc"))

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [36]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=str(dataset_dir_gdp).replace(".nc", "_CF.nc"), working_dir=CF_dir)

<IPython.core.display.Javascript object>

In [37]:
# save altered CF compliancy
save_compliancy(
    cap,
    testfile=str(dataset_dir_gdp).replace(".nc", "_CF.nc"),
    working_dir=CF_dir,
)

{'P:\\1000545-054-globalbeaches\\15_GlobalCoastalAtlas\\datasets\\06_Worldgdp\\world_gdp_CF.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

### write data to Zarr files

In [38]:
# export to zarr in write mode (to overwrite if exists)
ds_gdp.to_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file), mode="w")

<IPython.core.display.Javascript object>

In [39]:
test = xr.open_zarr(dataset_dir.joinpath("%s.zarr" % dataset_out_file))

<IPython.core.display.Javascript object>

In [40]:
test

<xarray.Dataset>
Dimensions:        (stations: 1709733)
Coordinates:
    continent      (stations) |S23 dask.array<chunksize=(26715,), meta=np.ndarray>
    country        (stations) |S40 dask.array<chunksize=(26715,), meta=np.ndarray>
    lat            (stations) float64 dask.array<chunksize=(53430,), meta=np.ndarray>
    lon            (stations) float64 dask.array<chunksize=(53430,), meta=np.ndarray>
    transect_geom  (stations) |S92 dask.array<chunksize=(13358,), meta=np.ndarray>
    transect_id    (stations) |S15 dask.array<chunksize=(53430,), meta=np.ndarray>
Dimensions without coordinates: stations
Data variables:
    gdp            (stations) float64 dask.array<chunksize=(53430,), meta=np.ndarray>
Attributes: (12/21)
    AUTHOR:              Kummu, M., Maija, T. & Guillaume, J.
    CITATION:            Kummu, M., Taka, M. & Guillaume, J. Gridded global d...
    COMMENT:             This dataset is modified from the original Kummu et ...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8
    DESCRIPTION:         The World Gridded GDP dataset contatins GDP per capi...
    ...                  ...
    SPATIAL_EXTENT:      [-180, -90, 180, 90]
    TAGS:                ['gdp', 'socio-economic']
    TEMPORAL_EXTENT:     [2015]
    TITLE:               World Gridded GDP
    TITLE_ABBREVIATION:  world_gdp
    UNITS:

<IPython.core.display.Javascript object>